# Two Point Convergence Study: Exploration

#### Imports

In [106]:
from os import getcwd, path
import tarfile

import warnings
warnings.filterwarnings("ignore")

In [107]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from typing import List
from ast import literal_eval

In [108]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

#### Utility

In [109]:
# color palette
viridis = ["#472c7a", "#2c718e", "#29ae80", "#b0dd31"]
viridis_light = ["rgba(71, 44, 122, 0.5)", "rgba(44, 113, 142, 0.5)", "rgba(41, 174, 128, 0.5)", "rgba(176, 221, 49, 0.5)"]

#alt_colors = ["#606060"] + [None]*6
#alt_colors[1::2] = list(reversed(viridis))[:-1]
#alt_colors[2::2] = list(reversed(viridis_light))[:-1]

alt_colors = ["#606060", "#29ae80", "rgba(44, 113, 142, 0.5)"]

In [110]:
def literal_eval_cols(data: DataFrame, cols: List[str]):
    for col_name in cols:
        data[col_name] = data.apply(lambda x: literal_eval(x[col_name]), axis=1)

### Relative Paths

In [111]:
data_dir = path.join(getcwd(), "data")
plot_dir = path.join(getcwd(), "plots")

### Load Ensembles

#### Raw Data

In [112]:
# loading the data

data_file_name = 'data_two_point_convergence_binned_sp_7_all_configs.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    re_data = pd.read_csv(path.join(data_dir, file_name))

else:
    re_data = pd.read_csv(path.join(data_dir,data_file_name))

print(re_data.columns)
re_data.shape

Index(['model_name', 'ds', 'n_sentence_pool', 'ds_infer_dens',
       'ds_n_consistent_complete_positions', 'account_penalties',
       'faithfulness_penalties', 'weight_account', 'weight_systematicity',
       'weight_faithfulness', 'init_coms', 'init_coms_dia_consistent',
       'fixed_point_coms', 'fixed_point_coms_consistent', 'fixed_point_theory',
       'fixed_point_dia_consistent', 'n_branches', 'fixed_points',
       'n_fixed_points', 'fp_coms_consistent', 'fp_union_consistent',
       'fixed_point_is_global_optimum', 'fixed_point_is_re_state',
       'fixed_point_is_full_re_state', 'global_optima', 'n_global_optima',
       'go_coms_consistent', 'go_union_consistent', 'go_full_re_state',
       'full_re_states', 'n_full_re_states', 'go_fixed_point',
       'fp_full_re_state', 'fp_global_optimum', 'configuration'],
      dtype='object')


(130000, 35)

In [113]:
# restrict df to individual simulation setups (fixed points are stored row-wise in lists anyway)
re_data.drop_duplicates(["weight_account", "weight_faithfulness", "weight_systematicity","ds","init_coms"], inplace=True)
re_data.shape

(130000, 35)

In [114]:
literal_eval_cols(re_data, ["fixed_points", 
                            #"global_optima", 
                            "fp_full_re_state", 
                            #"go_full_re_state"
                           ])

In [115]:
re_data["configuration"] = re_data.apply(lambda row: (row["weight_account"], 
                                                      row["weight_systematicity"], 
                                                      row["weight_faithfulness"]), axis=1)

#### Preprocessed Data

In [116]:
# loading the data

data_file_name = 'preprocessed_data_two_point_convergence_binned_sp_7_all_configs2.csv.tar.gz'

if data_file_name[data_file_name.find('.'):len(data_file_name)] == '.csv.tar.gz':
    with tarfile.open(path.join(data_dir,data_file_name)) as tar:
        for tarinfo in tar:
            file_name = tarinfo.name
        tar.extractall(data_dir)
    result_df = pd.read_csv(path.join(data_dir, file_name))

else:
    result_df = pd.read_csv(path.join(data_dir,data_file_name))
    
print(result_df.columns)
result_df.shape

Index(['ds', 'n', 'infer_dens', 'init_coms_agreement', 'init_coms_compat',
       'init_coms_size', 'so_coms_group_agreement', 'so_coms_group_compat',
       'so_coms_size', 'so_thes_group_agreement', 'so_thes_group_compat',
       'so_coms_compat_list', 'so_coms_simil_list', 'so_thes_size',
       'configuration', 'fp_coms_group_agreement', 'fp_coms_group_compat',
       'fp_coms_compat_list', 'fp_coms_simil_list', 'fp_coms_size',
       'fp_thes_group_agreement', 'fp_thes_group_compat', 'fp_thes_size'],
      dtype='object')


(17063, 23)

In [117]:
literal_eval_cols(result_df, ["configuration", "fp_coms_compat_list", "fp_coms_simil_list"])

## General Information

In [118]:
re_data["ds"].nunique()

13000

In [119]:
print("Mean inferenital density:", re_data["ds_infer_dens"].mean())

Mean inferenital density: 0.2982112675738665


In [120]:
re_data["ds_infer_dens"].describe()

count    130000.000000
mean          0.298211
std           0.078157
min           0.152752
25%           0.239725
50%           0.288986
75%           0.345005
max           0.487862
Name: ds_infer_dens, dtype: float64

### Fixed Points

In [121]:
# fixed points

fp_df = re_data[['n_sentence_pool', "ds", "configuration", "init_coms", "fixed_points", "fp_full_re_state"]]

In [122]:
# average number of fixed points reached from individual simulation setups
fp_df.groupby("configuration")["fixed_points"].agg(lambda x: sum(len(y) for y in x)/len(x))

configuration
(0.35, 0.55, 0.1)    2.140385
(0.46, 0.1, 0.44)    1.654808
(0.55, 0.2, 0.25)    1.896692
(0.55, 0.35, 0.1)    1.585154
(0.7, 0.2, 0.1)      1.599115
Name: fixed_points, dtype: float64

In [123]:
# total number of fixed points
fp_df.groupby("configuration")["fixed_points"].agg(lambda x: sum(len(y) for y in x))

configuration
(0.35, 0.55, 0.1)    55650
(0.46, 0.1, 0.44)    43025
(0.55, 0.2, 0.25)    49314
(0.55, 0.35, 0.1)    41214
(0.7, 0.2, 0.1)      41577
Name: fixed_points, dtype: int64

In [124]:
# total number of full RE fixed points
fp_df.groupby("configuration")["fp_full_re_state"].agg(lambda x: sum(sum(y) for y in x))

configuration
(0.35, 0.55, 0.1)    13106
(0.46, 0.1, 0.44)    23185
(0.55, 0.2, 0.25)    15103
(0.55, 0.35, 0.1)    17979
(0.7, 0.2, 0.1)      11478
Name: fp_full_re_state, dtype: int64

In [125]:
# skwewness > 1: highly skewed
fp_df["fixed_points"].map(len).skew()

6.1858179381311915

### Full RE Fixed Points

In [126]:
fp_df["fixed_points_lengths"] = fp_df.apply(lambda row: [(len(fp[0]), len(fp[1])) 
                                                                       for fp in row["fixed_points"]], axis=1)

In [127]:
# explode fixed point information
ex_fp_df = fp_df.set_index(['n_sentence_pool', "ds", "configuration", "init_coms"]).apply(pd.Series.explode).reset_index()
ex_fp_df.shape

(230780, 7)

In [128]:
ex_fp_df.shape

(230780, 7)

In [129]:
# fixed points
ex_fp_df.groupby("configuration")["fixed_points_lengths"].size()

configuration
(0.35, 0.55, 0.1)    55650
(0.46, 0.1, 0.44)    43025
(0.55, 0.2, 0.25)    49314
(0.55, 0.35, 0.1)    41214
(0.7, 0.2, 0.1)      41577
Name: fixed_points_lengths, dtype: int64

In [130]:
# restrict to full RE fixed points
ex_fp_df = ex_fp_df[ex_fp_df["fp_full_re_state"]]
ex_fp_df.shape

(80851, 7)

In [131]:
# full RE fixed points
ex_fp_df.groupby("configuration")["fixed_points_lengths"].size()

configuration
(0.35, 0.55, 0.1)    13106
(0.46, 0.1, 0.44)    23185
(0.55, 0.2, 0.25)    15103
(0.55, 0.35, 0.1)    17979
(0.7, 0.2, 0.1)      11478
Name: fixed_points_lengths, dtype: int64

In [132]:
# restrict further to non-trivial full RE fixed points
#ex_fp_df = ex_fp_df[ex_fp_df["fixed_points_lengths"]!=(1,1)]
#ex_fp_df.shape

In [133]:
# full RE fixed points
ex_fp_df.groupby("configuration")["fixed_points_lengths"].size()

configuration
(0.35, 0.55, 0.1)    13106
(0.46, 0.1, 0.44)    23185
(0.55, 0.2, 0.25)    15103
(0.55, 0.35, 0.1)    17979
(0.7, 0.2, 0.1)      11478
Name: fixed_points_lengths, dtype: int64

In [134]:
# number of full RE fixed points per simulation setup
ex_fp_df.groupby(["configuration", "ds", "init_coms"])["fixed_points"].size().describe()

count    61838.000000
mean         1.307465
std          0.737057
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         16.000000
Name: fixed_points, dtype: float64

In [135]:
# number of simulation setups that result in more than one full RE fixed point
n_fp_df = ex_fp_df.groupby(["configuration", "ds", "init_coms"])["fixed_points"].size().reset_index()
n_fp_df[n_fp_df["fixed_points"]>1].shape

(13130, 4)

In [136]:
# convert fixed points to sorted tuples
ex_fp_df["fixed_points_tuple"] = ex_fp_df.apply(lambda row: (tuple(sorted(row["fixed_points"][0])), 
                                                             tuple(sorted(row["fixed_points"][1]))), axis=1)

In [137]:
ex_fp_df.head()

,n_sentence_pool,ds,configuration,init_coms,fixed_points,fp_full_re_state,fixed_points_lengths,fixed_points_tuple
0,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.55, 0.35, 0.1)","{-7, -5, -4, -3, -2}","({-1, 6}, {2, 6, -5, -4, -3, -1})",True,"(2, 6)","((-1, 6), (-5, -4, -3, -1, 2, 6))"
2,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.55, 0.2, 0.25)","{-7, -5, -4, -3, -2}","({-7, -1, 6}, {2, 6, -7, -5, -4, -3, -1})",True,"(3, 7)","((-7, -1, 6), (-7, -5, -4, -3, -1, 2, 6))"
3,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.46, 0.1, 0.44)","{-7, -5, -4, -3, -2}","({-7, -1, 6}, {2, 6, -7, -5, -4, -3, -1})",True,"(3, 7)","((-7, -1, 6), (-7, -5, -4, -3, -1, 2, 6))"
4,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.55, 0.35, 0.1)","{-7, -4, -3, -2}","({-3}, {2, -4, -3})",True,"(1, 3)","((-3,), (-4, -3, 2))"
7,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.46, 0.1, 0.44)","{-7, -4, -3, -2}","({-7, -3}, {-7, 2, -4, -3})",True,"(2, 4)","((-7, -3), (-7, -4, -3, 2))"


In [138]:
# convert fixed points to tuples
#ex_fp_df["fixed_point_tuple"] = ex_fp_df.apply(lambda row: (tuple(sorted(row["fixed_points"][0])), 
#                                                             tuple(sorted(row["fixed_points"][1]))), axis=1)

In [139]:
# count the number of unique initial commitments that yield full RE fixed point
# per dialectical structure and configuration
ex_fp_df_grouped = ex_fp_df.groupby(["ds", "configuration"])["init_coms"].nunique().reset_index()


# select pairs of simulation setups that both yield at least one full RE fixed point
full_re_fp_df = fp_df.merge(ex_fp_df_grouped[ex_fp_df_grouped["init_coms"]==2][["ds", "configuration"]], 
                         on=["ds", "configuration"], how="inner")

# only keep full RE fixed points
full_re_fp_df["full_re_fixed_points"] = full_re_fp_df.apply(lambda row: [pair[0] 
                                                                         for pair in zip(row["fixed_points"], row["fp_full_re_state"]) if pair[1]], axis=1)

# only keep lengths of full RE fixed points
full_re_fp_df["full_re_fixed_points_lengths"] = full_re_fp_df.apply(lambda row: [pair[0] for pair in zip(row["fixed_points_lengths"], 
                                                                                                         row["fp_full_re_state"]) if pair[1]], axis=1)
# drop columns
full_re_fp_df.drop(["fixed_points","fixed_points_lengths", "fp_full_re_state"], axis=1, inplace=True)



paired_ex_fp_df = full_re_fp_df.set_index(['n_sentence_pool', "ds", "configuration", "init_coms"]).apply(pd.Series.explode).reset_index()
print(paired_ex_fp_df.shape)


# convert fixed points to sorted tuples
paired_ex_fp_df["full_re_fixed_points_tuple"] = paired_ex_fp_df.apply(lambda row: (tuple(sorted(row["full_re_fixed_points"][0])), 
                                                             tuple(sorted(row["full_re_fixed_points"][1]))), axis=1)

(44649, 6)


In [201]:
full_re_fp_df.groupby("configuration")["ds"].nunique()

configuration
(0.35, 0.55, 0.1)    2515
(0.46, 0.1, 0.44)    6147
(0.55, 0.2, 0.25)    2819
(0.55, 0.35, 0.1)    3940
(0.7, 0.2, 0.1)      1642
Name: ds, dtype: int64

In [141]:
# number of paired simulation setups that result in more than one full RE fixed point
n_paired_fp_df = paired_ex_fp_df.groupby(["configuration", "ds",])["full_re_fixed_points_tuple"].nunique().reset_index()
n_paired_fp_df[n_paired_fp_df["full_re_fixed_points_tuple"]>1].shape

(14430, 3)

In [142]:
# grouped by configuration
n_fp_df[n_fp_df["fixed_points"]>1].groupby("configuration")["fixed_points"].size()

configuration
(0.35, 0.55, 0.1)    1824
(0.46, 0.1, 0.44)    3816
(0.55, 0.2, 0.25)    2431
(0.55, 0.35, 0.1)    3181
(0.7, 0.2, 0.1)      1878
Name: fixed_points, dtype: int64

In [143]:
# paired simulation setups grouped by configuration
n_paired_fp_df[n_paired_fp_df["full_re_fixed_points_tuple"]>1].groupby("configuration")["full_re_fixed_points_tuple"].size()

configuration
(0.35, 0.55, 0.1)    1900
(0.46, 0.1, 0.44)    5552
(0.55, 0.2, 0.25)    2379
(0.55, 0.35, 0.1)    3292
(0.7, 0.2, 0.1)      1307
Name: full_re_fixed_points_tuple, dtype: int64

In [144]:
sub_n_fp_df = n_fp_df[n_fp_df["configuration"]==(0.35, 0.55, 0.10)]
sub_n_paired_fp_df = n_paired_fp_df[n_paired_fp_df["configuration"]==(0.35, 0.55, 0.10)]

In [145]:
sub_n_fp_df.shape

(10505, 4)

In [202]:
sub_n_paired_fp_df.shape

(1642, 3)

In [146]:
sub_n_fp_df.groupby("fixed_points")["init_coms"].size().reset_index()

,fixed_points,init_coms
0,1,8681
1,2,1233
2,3,463
3,4,88
4,5,35
5,6,2
6,7,1
7,10,1
8,14,1


In [147]:
sub_n_paired_fp_df.groupby("full_re_fixed_points_tuple")["ds"].size().reset_index()

,full_re_fixed_points_tuple,ds
0,1,615
1,2,1300
2,3,381
3,4,150
4,5,44
5,6,17
6,7,5
7,8,2
8,14,1


In [148]:
n=0
count = 0
conv = 0

for configuration in [(0.35,0.55, 0.1)]: #full_re_fp_df["configuration"].unique():
    sub_full_re_fp_df = full_re_fp_df[full_re_fp_df["configuration"]==configuration]
    
    for ds in sub_full_re_fp_df["ds"].unique():
        sub_sub_full_re_fp_df = sub_full_re_fp_df[sub_full_re_fp_df["ds"]==ds]

        n+=1
           
        # initial coms 
        ic1, ic2 = sub_sub_full_re_fp_df["init_coms"].unique()
        
        
        fp1 = [ (tuple(sorted(fp[0])), 
                 tuple(sorted(fp[1]))) for fps in sub_sub_full_re_fp_df[sub_sub_full_re_fp_df["init_coms"]==ic1]["full_re_fixed_points"] for fp in fps]
        
        
        
        fp2 = [ (tuple(sorted(fp[0])), 
                 tuple(sorted(fp[1]))) for fps in sub_sub_full_re_fp_df[sub_sub_full_re_fp_df["init_coms"]==ic2]["full_re_fixed_points"] for fp in fps]
        if len(fp1)==1 and len(fp2)==1:
            count+=1
            if fp1[0]==fp2[0]:
                conv+=1

print(n, count, conv)
            

2515 1812 615


In [194]:
output_table = pd.DataFrame()

for configuration in n_fp_df["configuration"].unique(): 
    
    table_row = {}
    table_row["configuration"] = configuration
    
    print(configuration)
    
    

    sub_n_fp_df = n_fp_df[n_fp_df["configuration"]==configuration]
    sub_n_paired_fp_df = n_paired_fp_df[n_paired_fp_df["configuration"]==configuration]


    # unpack weights
    alpha_a, alpha_s, alpha_f = configuration

    # create strings for annotation and filename
    annotation_text = r"$(\alpha_{A}, \alpha_{S}, \alpha_{F}) = " + "({},\ {},\ {})$".format(alpha_a, alpha_s, alpha_f)
    file_name = "intra_inter_convergence2_{}_{}_{}.png".format(str(alpha_a)[2:], str(alpha_s)[2:], str(alpha_f)[2:])


    fig = go.Figure()   

    # intrapersonal
    l = len(sub_n_fp_df)
    rel_conv = len(sub_n_fp_df[sub_n_fp_df["fixed_points"]<=1])/l
    rel_non_conv = len(sub_n_fp_df[sub_n_fp_df["fixed_points"]>1])/l
    
    print(l, len(sub_n_fp_df[sub_n_fp_df["fixed_points"]<=1]), rel_conv)
    
    table_row["rel_conv_intra"] = round(rel_conv, 2)

    intra = "Intrapersonal convergence<br>({} individual setups)".format(l)

    fig.add_trace(go.Bar(y=[rel_conv],
                         x=[intra],
                         text= str(round(rel_conv,2)),
                         textposition="auto",
                         marker_color=viridis[2],
                         name="unique output"
                        ))

    fig.add_trace(go.Bar(y=[rel_non_conv],
                         x=[intra],
                         text= str(round(rel_non_conv,2)),
                         textposition="auto",
                         marker_color="lightgray",
                         name="multiple outputs"
                        ))

    #interpersonal

    

    sub_paired_ex_fp_df = paired_ex_fp_df[paired_ex_fp_df["configuration"]==configuration]

    n = len(sub_paired_ex_fp_df["ds"].unique())
    # pairs of simulation setups that both yield a unique fixed point
    df1 = sub_paired_ex_fp_df.groupby(["ds","init_coms"])["full_re_fixed_points"].size().reset_index().groupby("ds")["full_re_fixed_points"].sum().reset_index()
    count = len(df1[df1["full_re_fixed_points"]==2])
    # pairs of simulation setups that yield the same unique fixed point
    df2 = sub_paired_ex_fp_df.groupby("ds")["full_re_fixed_points_tuple"].nunique().reset_index()
    conv = len(df2[df2["full_re_fixed_points_tuple"]==1])
    
    print(count, conv, conv/count)
    
    table_row["rel_conv_inter"] = round(conv/count, 2)


    inter = "Interpersonal convergence<br>({} paired setups)".format(count)

    fig.add_trace(go.Bar(y=[conv/count],
                         x=[inter],
                         text= str(round(conv/count,2)),
                         textposition="auto",
                         marker_color=viridis[2],
                         name="Unique output",
                         showlegend=False
                        ))

    fig.add_trace(go.Bar(y=[(count-conv)/count],
                         x=[inter],
                         text= str(round((count-conv)/count,2)),
                         textposition="auto",
                         marker_color="lightgray",
                         name="Multiple outputs",
                         showlegend=False
                        ))




    fig.update_layout(barmode="stack")
    fig.update_layout(template="plotly_white")

    fig.update_traces(width=0.3)

    fig.update_xaxes(title="", showticklabels=True)
    fig.update_yaxes(title="Relative share", showticklabels = True)
    fig.update_layout(legend_title_text="")

    #fig.update_layout(width=600)

    fig.update_layout(legend=dict(
            orientation="v",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1.0))

    fig.show()
    
    output_table = output_table.append(table_row, ignore_index=True)

    pio.write_image(fig, path.join(plot_dir, file_name), scale=2)

(0.35, 0.55, 0.1)
10505 8681 0.8263683960019038
1812 615 0.3394039735099338


(0.46, 0.1, 0.44)
17634 13818 0.7835998638992855
3958 595 0.15032844871147044


(0.55, 0.2, 0.25)
11654 9223 0.7914020937017333
1862 440 0.23630504833512353


(0.55, 0.35, 0.1)
13621 10440 0.7664635489317965
2444 648 0.265139116202946


(0.7, 0.2, 0.1)
8424 6546 0.7770655270655271
1081 335 0.3098982423681776


In [150]:
output_table

,configuration,rel_conv_intra,rel_conv_inter
0,"(0.35, 0.55, 0.1)",0.83,0.34
1,"(0.46, 0.1, 0.44)",0.78,0.15
2,"(0.55, 0.2, 0.25)",0.79,0.24
3,"(0.55, 0.35, 0.1)",0.77,0.27
4,"(0.7, 0.2, 0.1)",0.78,0.31


In [151]:
print(output_table.set_index("configuration").to_latex(float_format="%.2f"))

\begin{tabular}{lrr}
\toprule
{} &  rel\_conv\_intra &  rel\_conv\_inter \\
configuration     &                 &                 \\
\midrule
(0.35, 0.55, 0.1) &            0.83 &            0.34 \\
(0.46, 0.1, 0.44) &            0.78 &            0.15 \\
(0.55, 0.2, 0.25) &            0.79 &            0.24 \\
(0.55, 0.35, 0.1) &            0.77 &            0.27 \\
(0.7, 0.2, 0.1)   &            0.78 &            0.31 \\
\bottomrule
\end{tabular}



In [ ]:
(0.35, 0.55, 0.10) &            0.83 &            0.34 \\
(0.46, 0.10, 0.44) &            0.78 &            0.15 \\
(0.55, 0.20, 0.25) &            0.79 &            0.24 \\
(0.55, 0.35, 0.10) &            0.77 &            0.27 \\
(0.70, 0.20, 0.10) &            0.78 &            0.31 \\

In [152]:
# count the number of unique initial commitments that yield full RE fixed point
# per dialectical structure and configuration
ex_fp_df_grouped = ex_fp_df.groupby(["ds", "configuration"])["init_coms"].nunique().reset_index()

In [153]:
# select pairs of simulation setups that both yield at least one full RE fixed point
full_re_fp_df = fp_df.merge(ex_fp_df_grouped[ex_fp_df_grouped["init_coms"]==2][["ds", "configuration"]], 
                         on=["ds", "configuration"], how="inner")
full_re_fp_df.shape

(34126, 7)

In [154]:
full_re_fp_df.head()

,n_sentence_pool,ds,configuration,init_coms,fixed_points,fp_full_re_state,fixed_points_lengths
0,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.55, 0.35, 0.1)","{-7, -5, -4, -3, -2}","[({-1, 6}, {2, 6, -5, -4, -3, -1})]",[True],"[(2, 6)]"
1,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.55, 0.35, 0.1)","{-7, -4, -3, -2}","[({-3}, {2, -4, -3})]",[True],"[(1, 3)]"
2,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.46, 0.1, 0.44)","{-7, -5, -4, -3, -2}","[({-7, -1, 6}, {2, 6, -7, -5, -4, -3, -1})]",[True],"[(3, 7)]"
3,7,"[[-2, 7, 1], [-4, 2], [5, 1], [3, 6, 5], [4, 3...","(0.46, 0.1, 0.44)","{-7, -4, -3, -2}","[({-7, -3}, {-7, 2, -4, -3})]",[True],"[(2, 4)]"
4,7,"[[7, -6, -2], [-5, -4, 2], [-3, -1, -6], [-1, ...","(0.46, 0.1, 0.44)","{7, -2, -4, -3, -1}","[({-4, -2, 7}, {3, 5, 6, 7, -2, -4, -1}), ({7,...","[True, True]","[(3, 7), (3, 7)]"


In [155]:
full_re_fp_df["fp_full_re_state"].map(sum).describe()

count    34126.000000
mean         1.308357
std          0.739749
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         16.000000
Name: fp_full_re_state, dtype: float64

In [156]:
full_re_fp_df["fp_full_re_state"].map(sum).skew()

4.203716038399439

In [157]:
# number of full RE fixed points grouped by weight configuration
full_re_fp_df.groupby(["configuration"])["fixed_points"].size().reset_index()

,configuration,fixed_points
0,"(0.35, 0.55, 0.1)",5030
1,"(0.46, 0.1, 0.44)",12294
2,"(0.55, 0.2, 0.25)",5638
3,"(0.55, 0.35, 0.1)",7880
4,"(0.7, 0.2, 0.1)",3284


In [158]:
# overall number of full RE fixed points reached from individual commitments
full_re_fp_df["fixed_points"].map(len).skew()

3.892038008433361

In [159]:
# average number of full RE fixed points reached from individual initial commitments per config
full_re_fp_df.groupby(["configuration"])["fixed_points"].agg(lambda x: sum(len(y) for y in x)/len(x))

configuration
(0.35, 0.55, 0.1)    1.254076
(0.46, 0.1, 0.44)    1.351554
(0.55, 0.2, 0.25)    1.365023
(0.55, 0.35, 0.1)    1.489213
(0.7, 0.2, 0.1)      1.462241
Name: fixed_points, dtype: float64

In [160]:
# average number of full RE fixed points reached from individual initial commitments per config
full_re_fp_df.groupby(["configuration"])["fp_full_re_state"].agg(lambda x: sum(sum(y) for y in x)/len(x))

configuration
(0.35, 0.55, 0.1)    1.234791
(0.46, 0.1, 0.44)    1.318123
(0.55, 0.2, 0.25)    1.306846
(0.55, 0.35, 0.1)    1.325381
(0.7, 0.2, 0.1)      1.346224
Name: fp_full_re_state, dtype: float64

In [161]:
# median and IQR number of full RE fixed points reached from individual initial commitments per config
full_re_fp_df.groupby(["configuration"])["fixed_points"].agg(lambda x: np.percentile([len(y) for y in x], [25, 50, 75]))

configuration
(0.35, 0.55, 0.1)    [1.0, 1.0, 1.0]
(0.46, 0.1, 0.44)    [1.0, 1.0, 1.0]
(0.55, 0.2, 0.25)    [1.0, 1.0, 1.0]
(0.55, 0.35, 0.1)    [1.0, 1.0, 2.0]
(0.7, 0.2, 0.1)      [1.0, 1.0, 2.0]
Name: fixed_points, dtype: object

## Compatibility

In [195]:
# relative share of compatibile input commitments
result_df.groupby(["configuration"])["init_coms_compat"].mean()

configuration
(0.35, 0.55, 0.1)    0.163817
(0.46, 0.1, 0.44)    0.148040
(0.55, 0.2, 0.25)    0.129833
(0.55, 0.35, 0.1)    0.079442
(0.7, 0.2, 0.1)      0.063337
Name: init_coms_compat, dtype: float64

In [163]:
# relative share of compatibile input commitments
result_df.groupby(["configuration"])["init_coms_compat"].mean()

configuration
(0.35, 0.55, 0.1)    0.163817
(0.46, 0.1, 0.44)    0.148040
(0.55, 0.2, 0.25)    0.129833
(0.55, 0.35, 0.1)    0.079442
(0.7, 0.2, 0.1)      0.063337
Name: init_coms_compat, dtype: float64

In [164]:
# initial compatibility
result_df["init_coms_compat"].describe()

count    17063.000000
mean         0.123366
std          0.328867
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: init_coms_compat, dtype: float64

In [165]:
# relative share of compatibile pairs of initial commitments
result_df["init_coms_compat"].sum()/result_df["init_coms_compat"].size

0.12336634823887944

In [166]:
# group compatibility
result_df["fp_coms_group_compat"].describe()

count    17063.000000
mean         0.322986
std          0.449216
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: fp_coms_group_compat, dtype: float64

In [167]:
# group compatibility via compatibility lists
result_df["fp_coms_compat_list"].map(lambda x: sum(x)/len(x)).describe()

count    17063.000000
mean         0.322986
std          0.449216
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: fp_coms_compat_list, dtype: float64

In [168]:
# relative share of compatible pairs/similarity via explosion
sub_result_df = result_df[["ds", "configuration", "init_coms_agreement", "fp_coms_compat_list", "fp_coms_simil_list"]]
ex_sub_result_df = sub_result_df.set_index(["ds", "configuration", "init_coms_agreement"]).apply(pd.Series.explode).reset_index()


In [169]:
# relative share of compatible pairs
ex_sub_result_df[ex_sub_result_df["configuration"]==(0.46, 0.10, 0.44)]["fp_coms_compat_list"].astype(float).describe()

count    11584.000000
mean         0.230145
std          0.420944
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: fp_coms_compat_list, dtype: float64

In [170]:
# similarity
ex_sub_result_df["fp_coms_simil_list"].astype(float).describe()

count    31590.000000
mean         0.577773
std          0.292852
min          0.000000
25%          0.357143
50%          0.571429
75%          0.857143
max          1.000000
Name: fp_coms_simil_list, dtype: float64

In [171]:
ex_sub_result_df.groupby("init_coms_agreement")["fp_coms_simil_list"].median().reset_index()

,init_coms_agreement,fp_coms_simil_list
0,0.071429,0.285714
1,0.142857,0.285714
2,0.214286,0.357143
3,0.285714,0.357143
4,0.357143,0.357143
5,0.428571,0.428571
6,0.500000,0.500000
7,0.571429,0.571429
8,0.642857,0.642857
9,0.714286,0.714286


In [172]:
# table: mean relative shares of compatible pairs of positions

result_df.groupby(["configuration"])[["init_coms_compat",
                                      "fp_coms_group_compat", 
                                      "so_coms_group_compat",
                                      #"fp_coms_pop_compat",
                                      #"so_coms_pop_compat",
                                     ]].mean().round(decimals=2)

,init_coms_compat,fp_coms_group_compat,so_coms_group_compat
configuration,,,
"(0.35, 0.55, 0.1)",0.16,0.54,0.33
"(0.46, 0.1, 0.44)",0.15,0.24,0.27
"(0.55, 0.2, 0.25)",0.13,0.31,0.28
"(0.55, 0.35, 0.1)",0.08,0.32,0.20
"(0.7, 0.2, 0.1)",0.06,0.33,0.26


In [173]:
result_df[["init_coms_compat", "fp_coms_compat_list"]].agg({"init_coms_compat":"mean", 
                                                            "fp_coms_compat_list": lambda x: sum(x.sum())/sum(x.map(len))}).reset_index()

,index,0
0,init_coms_compat,0.123366
1,fp_coms_compat_list,0.312789


In [203]:
result_df.groupby(["configuration"])["ds"].nunique()

configuration
(0.35, 0.55, 0.1)    2515
(0.46, 0.1, 0.44)    6147
(0.55, 0.2, 0.25)    2819
(0.55, 0.35, 0.1)    3940
(0.7, 0.2, 0.1)      1642
Name: ds, dtype: int64

In [204]:
configuration = (0.55, 0.35, 0.10)

df = result_df[result_df["configuration"]==configuration]

dff = df[["init_coms_compat", "fp_coms_compat_list"]].set_index("init_coms_compat").apply(pd.Series.explode).reset_index()

print(len(df), len(dff))

print(len(dff[(dff["init_coms_compat"]>0) & (dff["fp_coms_compat_list"]>0)]))
print(len(dff[(dff["init_coms_compat"]>0) & (dff["fp_coms_compat_list"]<1)]))
print(len(dff[(dff["init_coms_compat"]<1) & (dff["fp_coms_compat_list"]>0)]))
print(len(dff[(dff["init_coms_compat"]<1) & (dff["fp_coms_compat_list"]<1)]))

3940 7368
428
89
1832
5019


In [205]:
# dataframe for table
fp_table_df = pd.DataFrame() 

# loop
for configuration in result_df["configuration"].unique():
    
    print(configuration)
    
    table_row = {}
    table_row["configuration"] = configuration
    
    # unpack weights
    alpha_a, alpha_s, alpha_f = configuration
    
    # create strings for annotation and filename
    annotation_text = r"$(\alpha_{A}, \alpha_{S}, \alpha_{F}) = " + "({},\ {},\ {})$".format(alpha_a, alpha_s, alpha_f)
    file_name = "compatibility_flow2_{}_{}_{}.png".format(str(alpha_a)[2:], str(alpha_s)[2:], str(alpha_f)[2:])



    df = result_df[result_df["configuration"]==configuration]

    dff = df[["init_coms_compat", "fp_coms_compat_list"]].set_index("init_coms_compat").apply(pd.Series.explode).reset_index()

    print(len(dff))
    
    
    ic1_fp1 = len(dff[(dff["init_coms_compat"]>0) & (dff["fp_coms_compat_list"]>0)])
    ic1_fp0 = len(dff[(dff["init_coms_compat"]>0) & (dff["fp_coms_compat_list"]<1)])
    ic0_fp1 = len(dff[(dff["init_coms_compat"]<1) & (dff["fp_coms_compat_list"]>0)])
    ic0_fp0 = len(dff[(dff["init_coms_compat"]<1) & (dff["fp_coms_compat_list"]<1)])
    
    print("fp1", len(dff[(dff["init_coms_compat"]>0) & (dff["fp_coms_compat_list"]>0)]) + 
         len(dff[(dff["init_coms_compat"]<1) & (dff["fp_coms_compat_list"]>0)]))
    

    rel_ic1 = round((ic1_fp1 + ic1_fp0) / (ic1_fp1 + ic1_fp0 + ic0_fp1 + ic0_fp0), 2)
    rel_fp1 = round((ic1_fp1 + ic0_fp1) / (ic1_fp1 + ic1_fp0 + ic0_fp1 + ic0_fp0), 2)

    rel_ic0 = round(1-rel_ic1, 2)
    rel_fp0 = round(1-rel_fp1, 2)
    
    table_row["fp_rel_ic1"] = rel_ic1
    table_row["fp_rel_fp1"] = rel_fp1
    fp_table_df = fp_table_df.append(table_row, ignore_index = True)
    
    
    print("rel_ic1_fp0: ", ic1_fp0/(ic1_fp0+ic1_fp1))
    print("rel_ic0_fp1: ", ic0_fp1/(ic0_fp0+ic0_fp1))

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 20,
          thickness = 100,
          line =  dict(color = "black", width = 0.0),
          label = ["", "", "", ""],
          color = ["#29ae80", "darkgray","#29ae80", "darkgray"]
        ),
        link = dict(
          source = [0, 1, 0, 1], # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = [2, 2, 3, 3],
          value =  [ic1_fp1, ic0_fp1, ic1_fp0, ic0_fp0],
          color = ["rgba(41, 174, 128, 0.4)", "rgba(211,211,211, 0.5)", "rgba(41, 174, 128, 0.4)", "rgba(211,211,211, 0.5)"]
      ))])

    # Annotations

    # Initial Commitments
    fig.add_annotation(dict(font=dict(color="black",size=14),
                                    showarrow=False,
                                    x=-0.1,
                                    y=0.5,
                                    text="Initial commitments",
                                    textangle=-90,
                                    xref="paper",
                                    yref="paper"))

    # Compatible Initial Commitments
    fig.add_annotation(dict(font=dict(color="black",size=14),
                                    showarrow=False,
                                    x=0.03,
                                    y=-0.12,
                                    text="compatible<br>" + str(rel_ic1),
                                    textangle=0,
                                    xref="paper",
                                    yref="paper"))

    # Incompatbile Initial Commitments
    fig.add_annotation(dict(font=dict(color="black",size=14),
                                    showarrow=False,
                                    x=0.01,
                                    y=1.12,
                                    text="incompatible<br>" + str(rel_ic0),
                                    textangle=0,
                                    xref="paper",
                                    yref="paper"))

    # Fixed Points
    fig.add_annotation(dict(font=dict(color="black",size=14),
                                    showarrow=False,
                                    x=1.1,
                                    y=0.5,
                                    text="Fixed point commitments",
                                    textangle=90,
                                    xref="paper",
                                    yref="paper"))

    # Consistent Fixed Points
    fig.add_annotation(dict(font=dict(color="black",size=14),
                                    showarrow=False,
                                    x=0.97,
                                    y=-0.12,
                                    text="compatible<br>" + str(rel_fp1),
                                    textangle=0,
                                    xref="paper",
                                    yref="paper"))

    # Incompatible Fixed Points
    fig.add_annotation(dict(font=dict(color="black",size=14),
                                    showarrow=False,
                                    x=0.99,
                                    y=1.12,
                                    text="incompatible<br>" + str(rel_fp0),
                                    textangle=0,
                                    xref="paper",
                                    yref="paper"))

    fig.update_layout(width=600,)

    #fig.update_layout(title_text="Basic Sankey Diagram")

    fig.show()

    pio.write_image(fig, path.join(plot_dir, file_name), scale=2)
    

(0.55, 0.35, 0.1)
7368
fp1 2260
rel_ic1_fp0:  0.172147001934236
rel_ic0_fp1:  0.2674062180703547


(0.46, 0.1, 0.44)
11584
fp1 2666
rel_ic1_fp0:  0.3073453608247423
rel_ic0_fp1:  0.15859250398724084


(0.55, 0.2, 0.25)
5182
fp1 1570
rel_ic1_fp0:  0.2496194824961948
rel_ic0_fp1:  0.23801104972375692


(0.35, 0.55, 0.1)
4140
fp1 2371
rel_ic1_fp0:  0.07046070460704607
rel_ic0_fp1:  0.49529688418577306


(0.7, 0.2, 0.1)
3316
fp1 1014
rel_ic1_fp0:  0.18072289156626506
rel_ic0_fp1:  0.2787301587301587


In [179]:
fp_table_df

,configuration,fp_rel_ic1,fp_rel_fp1
0,"(0.55, 0.35, 0.1)",0.07,0.31
1,"(0.46, 0.1, 0.44)",0.13,0.23
2,"(0.55, 0.2, 0.25)",0.13,0.30
3,"(0.35, 0.55, 0.1)",0.18,0.57
4,"(0.7, 0.2, 0.1)",0.05,0.31


In [180]:
print(fp_table_df.set_index("configuration").to_latex(float_format="%.2f"))

\begin{tabular}{lrr}
\toprule
{} &  fp\_rel\_ic1 &  fp\_rel\_fp1 \\
configuration     &             &             \\
\midrule
(0.55, 0.35, 0.1) &        0.07 &        0.31 \\
(0.46, 0.1, 0.44) &        0.13 &        0.23 \\
(0.55, 0.2, 0.25) &        0.13 &        0.30 \\
(0.35, 0.55, 0.1) &        0.18 &        0.57 \\
(0.7, 0.2, 0.1)   &        0.05 &        0.31 \\
\bottomrule
\end{tabular}



In [71]:
ex_fp_df = fp_df.set_index(['n_sentence_pool', "ds", "configuration", "init_coms"]).apply(pd.Series.explode).reset_index()
print(ex_fp_df.shape)
#ex_go_df = go_df.set_index(['n_sentence_pool', "ds", "configuration", "init_coms"]).apply(pd.Series.explode).reset_index()
#print(ex_go_df.shape)

(230780, 7)


## Initial and Final Agreement (Similarity)

In [183]:
result_df.groupby(
    ["configuration"])[["init_coms_agreement",
                        "fp_coms_group_agreement",
                        #"so_coms_group_agreement",
                        #"fp_coms_pop_agreement",
                        #"so_coms_pop_agreement"
                       ]].median()

,init_coms_agreement,fp_coms_group_agreement
configuration,,
"(0.35, 0.55, 0.1)",0.642857,0.642857
"(0.46, 0.1, 0.44)",0.571429,0.500000
"(0.55, 0.2, 0.25)",0.571429,0.571429
"(0.55, 0.35, 0.1)",0.428571,0.500000
"(0.7, 0.2, 0.1)",0.571429,0.571429


In [181]:
result_df.query("init_coms_agreement < 0.4").groupby(
    ["configuration"])[["init_coms_agreement",
                        "fp_coms_group_agreement",
                        "so_coms_group_agreement",
                        #"fp_coms_pop_agreement",
                        #"so_coms_pop_agreement"
                       ]].mean()

,init_coms_agreement,fp_coms_group_agreement,so_coms_group_agreement
configuration,,,
"(0.35, 0.55, 0.1)",0.217318,0.400232,0.339319
"(0.46, 0.1, 0.44)",0.228198,0.303256,0.313227
"(0.55, 0.2, 0.25)",0.225275,0.310783,0.310748
"(0.55, 0.35, 0.1)",0.200384,0.427540,0.369044
"(0.7, 0.2, 0.1)",0.212107,0.381415,0.375776


In [73]:
result_df.query("init_coms_agreement < 0.4").groupby(["configuration"])[
    ["init_coms_agreement", 
     "fp_coms_simil_list"]].agg({"init_coms_agreement": "mean", 
                                 "fp_coms_simil_list": lambda x: sum(x.sum())/sum(x.map(len))}).round(decimals=2).reset_index()

,configuration,init_coms_agreement,fp_coms_simil_list
0,"(0.35, 0.55, 0.1)",0.22,0.40
1,"(0.46, 0.1, 0.44)",0.23,0.31
2,"(0.55, 0.2, 0.25)",0.23,0.31
3,"(0.55, 0.35, 0.1)",0.20,0.44
4,"(0.7, 0.2, 0.1)",0.21,0.41


In [74]:
result_df.query("init_coms_agreement >= 0.4 & init_coms_agreement <= 0.6").groupby(
    ["configuration"])[["init_coms_agreement",
                        "fp_coms_group_agreement",
                        "so_coms_group_agreement",
                        #"fp_coms_pop_agreement",
                        #"so_coms_pop_agreement"
                       ]].mean()

,init_coms_agreement,fp_coms_group_agreement,so_coms_group_agreement
configuration,,,
"(0.35, 0.55, 0.1)",0.507235,0.590358,0.532786
"(0.46, 0.1, 0.44)",0.501960,0.489117,0.514515
"(0.55, 0.2, 0.25)",0.500000,0.507460,0.518147
"(0.55, 0.35, 0.1)",0.500552,0.573848,0.538190
"(0.7, 0.2, 0.1)",0.502723,0.547076,0.538797


In [75]:
result_df.query("init_coms_agreement >= 0.4 & init_coms_agreement <= 0.6").groupby(["configuration"])[
    ["init_coms_agreement", 
     "fp_coms_simil_list"]].agg({"init_coms_agreement": "mean", 
                                 "fp_coms_simil_list": lambda x: sum(x.sum())/sum(x.map(len))}).round(decimals=2).reset_index()

,configuration,init_coms_agreement,fp_coms_simil_list
0,"(0.35, 0.55, 0.1)",0.51,0.63
1,"(0.46, 0.1, 0.44)",0.50,0.50
2,"(0.55, 0.2, 0.25)",0.50,0.53
3,"(0.55, 0.35, 0.1)",0.50,0.58
4,"(0.7, 0.2, 0.1)",0.50,0.58


In [76]:
result_df.query("init_coms_agreement > 0.6").groupby(
    ["configuration"])[["init_coms_agreement",
                        "fp_coms_group_agreement",
                        "so_coms_group_agreement",
                        #"fp_coms_pop_agreement",
                        #"so_coms_pop_agreement"
                       ]].mean()

,init_coms_agreement,fp_coms_group_agreement,so_coms_group_agreement
configuration,,,
"(0.35, 0.55, 0.1)",0.794055,0.802921,0.763661
"(0.46, 0.1, 0.44)",0.791319,0.758083,0.774760
"(0.55, 0.2, 0.25)",0.794102,0.763676,0.760959
"(0.55, 0.35, 0.1)",0.795949,0.769705,0.757845
"(0.7, 0.2, 0.1)",0.799578,0.787831,0.747627


In [77]:
result_df.query("init_coms_agreement > 0.6").groupby(["configuration"])[
    ["init_coms_agreement", 
     "fp_coms_simil_list"]].agg({"init_coms_agreement": "mean", 
                                 "fp_coms_simil_list": lambda x: sum(x.sum())/sum(x.map(len))}).round(decimals=2).reset_index()

,configuration,init_coms_agreement,fp_coms_simil_list
0,"(0.35, 0.55, 0.1)",0.79,0.80
1,"(0.46, 0.1, 0.44)",0.79,0.75
2,"(0.55, 0.2, 0.25)",0.79,0.76
3,"(0.55, 0.35, 0.1)",0.80,0.76
4,"(0.7, 0.2, 0.1)",0.80,0.77


In [186]:
simil_table = result_df.groupby(["configuration"])[
    ["init_coms_agreement", 
     "fp_coms_simil_list"]].agg({"init_coms_agreement": "mean", 
                                 "fp_coms_simil_list": lambda x: sum(x.sum())/sum(x.map(len))}).round(decimals=2).reset_index()

simil_table

,configuration,init_coms_agreement,fp_coms_simil_list
0,"(0.35, 0.55, 0.1)",0.59,0.67
1,"(0.46, 0.1, 0.44)",0.54,0.55
2,"(0.55, 0.2, 0.25)",0.56,0.57
3,"(0.55, 0.35, 0.1)",0.46,0.57
4,"(0.7, 0.2, 0.1)",0.53,0.59


In [187]:
print(simil_table.set_index("configuration").to_latex(float_format="%.2f"))

\begin{tabular}{lrr}
\toprule
{} &  init\_coms\_agreement &  fp\_coms\_simil\_list \\
configuration     &                      &                     \\
\midrule
(0.35, 0.55, 0.1) &                 0.59 &                0.67 \\
(0.46, 0.1, 0.44) &                 0.54 &                0.55 \\
(0.55, 0.2, 0.25) &                 0.56 &                0.57 \\
(0.55, 0.35, 0.1) &                 0.46 &                0.57 \\
(0.7, 0.2, 0.1)   &                 0.53 &                0.59 \\
\bottomrule
\end{tabular}



In [ ]:
(0.35, 0.55, 0.10) &                 0.59 &                0.67 \\
(0.46, 0.10, 0.44) &                 0.54 &                0.55 \\
(0.55, 0.20, 0.25) &                 0.56 &                0.57 \\
(0.55, 0.35, 0.10) &                 0.46 &                0.57 \\
(0.70, 0.20, 0.10) &                 0.53 &                0.59 \\

In [190]:
# Boxplots grouped by initial similarity with diagonal line

color_map = {"go": viridis[3], "fp": viridis[2], "so": viridis[1]}

for configuration in result_df["configuration"].unique():

    #configuration = (0.35, 0.55, 0.1)
    print(configuration)

    # unpack weights
    alpha_a, alpha_s, alpha_f = configuration

    # create strings for annotation and filename
    annotation_text = r"$(\alpha_{A}, \alpha_{S}, \alpha_{F}) = " + "({},\ {},\ {})$".format(alpha_a, alpha_s, alpha_f)
    file_name = "input_output_similarity_box_{}_{}_{}.png".format(str(alpha_a)[2:], str(alpha_s)[2:], str(alpha_f)[2:])



    mdff = result_df[(result_df["configuration"]==configuration)][["init_coms_agreement", "fp_coms_group_agreement"]]

    test_df = result_df[(result_df["configuration"]==configuration)][["init_coms_agreement", "fp_coms_simil_list"]]
    ex_test_df = test_df.set_index("init_coms_agreement").apply(pd.Series.explode).reset_index()

    fig= px.box(ex_test_df,
               x="init_coms_agreement",
               y="fp_coms_simil_list",
               notched=True,

               )

    fig.update_traces(orientation='v', boxmean=False)


    fig.update_traces(marker_color=color_map["fp"],
                      name="Full RE fixed point commitments")

    marker_color=color_map["fp"]

    fig.update_layout(xaxis = dict(tickmode = 'array',
                                    tickvals = [round(t,2) for t in sorted(mdff["init_coms_agreement"].unique())]),
                                  )


    # diagonal line
    fig.add_shape(type="line",
                  xref="x", yref="y",
                  x0=0, y0=0, x1=1, y1=1,
                  line=dict(color="gray", dash="dot"))
    #
    #    
    #
    fig.update_traces(showlegend=True)
    #fig.update_xaxes(showticklabels=False)
    #
    fig.update_layout(template="plotly_white",
                      #height=600,
                      #width=650
                      )
    #    
    #fig.update_layout(title="Similarity of Output Binned by Initial Commitments Similarity")
    #

    fig.update_yaxes(title="Output similarity")
    fig.update_xaxes(title="Initial similarity")

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="left",
        x=0.0
    ))


    #fig.add_annotation(dict(font=dict(color="black",size=12),
    #                                showarrow=False,
    #                                x=-0.02,
    #                                y=1.04,
    #                                text=annotation_text,
    #                                textangle=0,
    #                                xref="paper",
    #                                yref="paper"))
    #
    #fig.update_layout(margin={"b":20, "t":75, "l":40, "r":20})

    fig.show()

    pio.write_image(fig, path.join(plot_dir, file_name), scale=2)

(0.55, 0.35, 0.1)


(0.46, 0.1, 0.44)


(0.55, 0.2, 0.25)


(0.35, 0.55, 0.1)


(0.7, 0.2, 0.1)
